In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -qq dash-bootstrap-components
!pip uninstall -qq polars -y
!pip install -qq polars==0.19.16
!pip install -qq pyngrok

In [ ]:
# import necessary libraries
import os
import gc
import pickle

import numpy as np
import pandas as pd
import polars as pl
import plotly.express as px

import json
import math
import pickle
import warnings
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional

import plotly
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objects as go
import plotly.express as px
import dash_bootstrap_components as dbc
from matplotlib.axes import Axes
from matplotlib.gridspec import GridSpec
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from pyngrok import ngrok
from dash import Dash, html, dcc, callback, Output, Input

warnings.simplefilter("ignore")
pl.Config.set_tbl_hide_dataframe_shape(True)
init_notebook_mode(connected=True)
sns.set_style("darkgrid")
colors = sns.color_palette('Set2')
import warnings 

warnings.filterwarnings("ignore")

import os
import gc
import datetime
import pickle 
import joblib

import numpy as np
import pandas as pd
import polars as pl
import plotly.express as px

from sklearn.ensemble import VotingRegressor
import lightgbm as lgb

import holidays

from scipy.signal import find_peaks, find_peaks, peak_widths

from scipy.stats import entropy

# Task Explaining
For each prediction_unit (unique identifier for the county, is_business, and product_type combination), we need to make predicitons of consumption or production of electricity on the next day.

Let's call a combination of county, is_business, is_consumption, product_type as a segment.We need to forecast each hour and each segment next day at 11:00:00. 
E.g. Imagine that we at 2023-05-27 11:00:00, and we have 
* revealed_targets (factual data for the previous day forecast),
* client (installed_capacity, eic_count, previous day), 
* historical_weather (previous day and current day before 11:00:00), 
* forecast_weather (current day and next 2 days), 
* electricity_prices (current day), 
* gas_prices (current day)

and we need to forecast the consumption or production each hour for each prediction_unit on day 2023-05-28.

# EDA (Exploratory Data Analysis) 

In [ ]:
# read data
RAW_DATA_PATH = Path("/kaggle/input/predict-energy-behavior-of-prosumers")
df_train = pd.read_csv(os.path.join(RAW_DATA_PATH, "train.csv"))
df_client = pd.read_csv(os.path.join(RAW_DATA_PATH, "client.csv"))
df_electricity_prices = pd.read_csv(os.path.join(RAW_DATA_PATH, "electricity_prices.csv"))
df_gas_prices = pd.read_csv(os.path.join(RAW_DATA_PATH, "gas_prices.csv"))
df_forecast_weather = pd.read_csv(os.path.join(RAW_DATA_PATH, "forecast_weather.csv"))
df_historical_weather = pd.read_csv(os.path.join(RAW_DATA_PATH, "historical_weather.csv"))
df_weather_station_to_county_mapping = pd.read_csv(os.path.join(RAW_DATA_PATH, "weather_station_to_county_mapping.csv"))

## train.csv
* train.csv includes 68 production units
* data datetime ranging from 2021.9.1 to 2023.5.31
* Production and Consumption Amount of Sample (Unit 0) visualization has been listed below

In [ ]:
df_train

In [ ]:
max(df_train["prediction_unit_id"])

In [ ]:
df_train.info()

In [ ]:
plt.subplot(2,1,1)
df_train_sample_pro = df_train[df_train['prediction_unit_id'] == 0][df_train['is_consumption'] == 0]
df_train_sample_pro['datetime']= pd.to_datetime(df_train_sample_pro['datetime'])
df_train_sample_con = df_train[df_train['prediction_unit_id'] == 0][df_train['is_consumption'] == 1]
df_train_sample_con['datetime']= pd.to_datetime(df_train_sample_con['datetime'])
plt.plot(df_train_sample_pro["datetime"],df_train_sample_pro["target"],label='production')
plt.plot(df_train_sample_con["datetime"],df_train_sample_con["target"],label='consumption')
plt.title('Production and Consumption Amount of Sample (Unit 0)')
plt.xlabel('Timestamp')
plt.ylabel('Production/Consumption')
plt.legend()
plt.subplot(2,1,2)
df_train_sample_pro = df_train[df_train['prediction_unit_id'] == 1][df_train['is_consumption'] == 0]
df_train_sample_pro['datetime']= pd.to_datetime(df_train_sample_pro['datetime'])
df_train_sample_con = df_train[df_train['prediction_unit_id'] == 1][df_train['is_consumption'] == 1]
df_train_sample_con['datetime']= pd.to_datetime(df_train_sample_con['datetime'])
plt.plot(df_train_sample_pro["datetime"],df_train_sample_pro["target"],label='production')
plt.plot(df_train_sample_con["datetime"],df_train_sample_con["target"],label='consumption')
plt.title('Production and Consumption Amount of Sample (Unit 1)')
plt.xlabel('Timestamp')
plt.ylabel('Production/Consumption')
plt.legend()

plt.tight_layout()  #adjust indent
plt.show()
# subplot

## client.csv
* data datetime ranging from 2021.9.1 to 2023.5.29
* EIC_count and installed_capacity of Sample (Unit 0) visualization has been listed below

In [ ]:
df_client

In [ ]:
plt.subplot(2,1,1)
df_client_sampe = df_client[df_client['county'] == 0][df_client['is_business'] == 0][df_client['product_type'] == 1]
df_client_sampe['date']= pd.to_datetime(df_client_sampe['date'])
plt.plot(df_client_sampe["date"],df_client_sampe["eic_count"],label='EIC_count')
plt.plot(df_client_sampe["date"],df_client_sampe["installed_capacity"],label='Installed_capacity')
plt.title('EIC_count and Installed_capacity of Sample (Unit 0)')
plt.xlabel('Date')
plt.ylabel('EIC_count and Installed_capacity')
plt.xticks(rotation=30)
plt.legend()

plt.subplot(2,1,2)
df_client_sampe = df_client[df_client['county'] == 0][df_client['is_business'] == 0][df_client['product_type'] == 2]
df_client_sampe['date']= pd.to_datetime(df_client_sampe['date'])
plt.plot(df_client_sampe["date"],df_client_sampe["eic_count"],label='EIC_count')
plt.plot(df_client_sampe["date"],df_client_sampe["installed_capacity"],label='Installed_capacity')
plt.title('EIC_count and Installed_capacity of Sample (Unit 1)')
plt.xlabel('Date')
plt.ylabel('EIC_count and Installed_capacity')
plt.xticks(rotation=30)
plt.legend()

plt.tight_layout()  #adjust indent
plt.show()
# subplot

## electricity_prices.csv
* data forecast date ranging from 2021.9.1 to 2023.5.30
* electricity prices euros_per_mwh visualization has been listed below

In [ ]:
df_electricity_prices

In [ ]:
#outlier dealing
df_electricity_prices[df_electricity_prices["euros_per_mwh"] == max(df_electricity_prices["euros_per_mwh"])]
df_electricity_prices.loc[df_electricity_prices["euros_per_mwh"] == max(df_electricity_prices["euros_per_mwh"]),"euros_per_mwh"] = 1000

In [ ]:
df_electricity_prices['origin_date']= pd.to_datetime(df_electricity_prices['origin_date'])
plt.plot(df_electricity_prices["origin_date"],df_electricity_prices["euros_per_mwh"],label='Euros_per_mwh')
plt.title('Electricity Prices')
plt.xlabel('Date')
plt.ylabel('Euros_per_mwh')
plt.xticks(rotation=15)
plt.legend()

## gas_prices.csv
* data forecast date ranging from 2021.9.1 to 2023.5.30
* gas prices lowest_price_per_mwh and highest_price_per_mwh visualization has been listed below

In [ ]:
df_gas_prices

In [ ]:
df_gas_prices['origin_date']= pd.to_datetime(df_electricity_prices['origin_date'])
plt.plot(df_gas_prices["origin_date"],df_gas_prices["lowest_price_per_mwh"],label='lowest_price_per_mwh')
plt.plot(df_gas_prices["origin_date"],df_gas_prices["highest_price_per_mwh"],label='highest_price_per_mwh')
plt.title('Gas Prices')
plt.xlabel('Date')
plt.ylabel('price_per_mwh')
plt.xticks(rotation=15)
plt.legend()

## historical_weather.csv
* date ranging from 2021.9.1 to 2023.5.30
* cloud cover and radiation changing of a location sample (Saaremaa: longitude == 22.2 latitude == 58.2 county == 10  2021-09) visualization has been listed blow

In [ ]:
df_historical_weather

In [ ]:
df_historical_weather.info()

In [ ]:
#sample Saaremaa longitude == 22.2 latitude == 58.2 county == 10  2021-09
df_historical_weather_sample = df_historical_weather[abs(df_historical_weather["longitude"] - 22.1) < 0.1][abs(df_historical_weather["latitude"] - 58.2) < 0.1][df_historical_weather["datetime"]<"2021-10-01"]
df_historical_weather_sample

In [ ]:
# cloudcover_total cloudcover_low cloudcover_mid cloudcover_high
plt.subplot(2,1,1)
df_historical_weather_sample = df_historical_weather[abs(df_historical_weather["longitude"] - 22.1) < 0.1][abs(df_historical_weather["latitude"] - 58.2) < 0.1][df_historical_weather["datetime"]<"2021-10-01"]
df_historical_weather_sample['datetime']= pd.to_datetime(df_historical_weather_sample['datetime'])
plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["cloudcover_total"],label='cloudcover_total', linewidth = 1)
# plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["cloudcover_low"],label='cloudcover_low', linewidth = 1)
# plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["cloudcover_mid"],label='cloudcover_mid', linewidth = 1)
# plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["cloudcover_high"],label='cloudcover_high', linewidth = 1)
plt.title('Historical Weather (Cloudcover) longitude: 22.1, latitude: 58.2')
plt.xlabel('Date')
plt.ylabel('Cloudcover')
plt.xticks(rotation=20)
plt.legend()

plt.subplot(2,1,2)
df_historical_weather_sample = df_historical_weather[abs(df_historical_weather["longitude"] - 22.7) < 0.1][abs(df_historical_weather["latitude"] - 58.8) < 0.1][df_historical_weather["datetime"]<"2021-10-01"]
df_historical_weather_sample['datetime']= pd.to_datetime(df_historical_weather_sample['datetime'])
plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["cloudcover_total"],label='cloudcover_total', linewidth = 1)
# plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["cloudcover_low"],label='cloudcover_low', linewidth = 1)
# plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["cloudcover_mid"],label='cloudcover_mid', linewidth = 1)
# plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["cloudcover_high"],label='cloudcover_high', linewidth = 1)
plt.title('Historical Weather (Cloudcover) longitude: 22.7, latitude: 58.8')
plt.xlabel('Date')
plt.ylabel('Cloudcover')
plt.xticks(rotation=20)
plt.legend()

plt.tight_layout()  #adjust indent
plt.show()
# subplot

In [ ]:
# shortwave_radiation direct_solar_radiation diffuse_radiation
plt.subplot(2,1,1)
df_historical_weather_sample = df_historical_weather[abs(df_historical_weather["longitude"] - 22.1) < 0.1][abs(df_historical_weather["latitude"] - 58.2) < 0.1][df_historical_weather["datetime"]<"2021-10-01"]
df_historical_weather_sample['datetime']= pd.to_datetime(df_historical_weather_sample['datetime'])
plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["shortwave_radiation"],label='shortwave_radiation')
plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["direct_solar_radiation"],label='direct_solar_radiation')
plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["diffuse_radiation"],label='diffuse_radiation')
plt.title('Historical Weather (Radiation) longitude: 22.1, latitude: 58.2')
plt.xlabel('Date')
plt.ylabel('Radiation')
plt.xticks(rotation=20)
plt.legend()

plt.subplot(2,1,2)
df_historical_weather_sample = df_historical_weather[abs(df_historical_weather["longitude"] - 22.7) < 0.1][abs(df_historical_weather["latitude"] - 58.8) < 0.1][df_historical_weather["datetime"]<"2021-10-01"]
df_historical_weather_sample['datetime']= pd.to_datetime(df_historical_weather_sample['datetime'])
plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["shortwave_radiation"],label='shortwave_radiation')
plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["direct_solar_radiation"],label='direct_solar_radiation')
plt.plot(df_historical_weather_sample['datetime'],df_historical_weather_sample["diffuse_radiation"],label='diffuse_radiation')
plt.title('Historical Weather (Radiation) longitude: 22.7, latitude: 58.8')
plt.xlabel('Date')
plt.ylabel('Radiation')
plt.xticks(rotation=20)
plt.legend()

plt.tight_layout()  #adjust indent
plt.show()
# subplot

In [ ]:
df_forecast_weather

In [ ]:
df_weather_station_to_county_mapping

In [ ]:
df_weather_station_to_county_mapping['size'] = 2
fig = px.scatter_mapbox(
    df_weather_station_to_county_mapping, 
    lat="latitude", 
    lon="longitude", 
    color="county",
    size="size",
    zoom=6,
    title='Weather Stations Locations'
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()